In [1]:
import os
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.boolean import BooleanOutputParser
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [3]:
load_dotenv("deepseek_api.env", override=True)

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")

In [5]:
# initialize deepseek api client
client = OpenAI(api_key=DeepSeek_API_KEY, base_url="https://api.deepseek.com")

# call deepseek api to generate answer
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant, please answer questions based on what the user asked."},
        {"role": "user", "content": "Hi there, please introduce yourself."}
    ],
)

# print the response result
print(response.choices[0].message.content)

Hello! I'm your helpful assistant, here to provide you with accurate information, answer your questions, assist with tasks, and offer support on a wide range of topics. Whether you need help with research, writing, problem-solving, or just a friendly chat, I'm here for you. Feel free to ask me anything!


In [6]:
model = init_chat_model(model="deepseek-chat", model_provider="deepseek")

In [7]:
question = "Hi there, please introduce yourself."

result = model.invoke(question)
print(result.content)

Hi! I'm DeepSeek-V3, your AI assistant created by DeepSeek. I'm here to help answer questions, provide insights, and assist with a wide range of topics—from general knowledge and coding to writing and productivity tips. I can analyze uploaded files, search the web when needed, and offer detailed, friendly conversations. Let me know how I can assist you today! 😊


In [8]:
result

AIMessage(content="Hi! I'm DeepSeek-V3, your AI assistant created by DeepSeek. I'm here to help answer questions, provide insights, and assist with a wide range of topics—from general knowledge and coding to writing and productivity tips. I can analyze uploaded files, search the web when needed, and offer detailed, friendly conversations. Let me know how I can assist you today! 😊", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 11, 'total_tokens': 92, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 11}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_08f168e49b_prod0820_fp8_kvcache', 'id': '88fe5738-0b40-491b-bdcd-ad651c8d0f9e', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--cbb5a2d9-7e69-43ca-b81a-be010e854375-0', usage_metadata={'input_tokens': 11, 'output_tokens': 81, 

In [9]:
# build a chain: model + parser
basic_qa_chain = model | StrOutputParser()

In [10]:
result = basic_qa_chain.invoke(question)
print(result)

Hi! I'm **DeepSeek-V3**, created by **DeepSeek**. I'm an AI assistant designed to help answer your questions, provide information, and assist with various tasks like writing, coding, learning, and more. I can process long documents (up to 128K context), support file uploads (PDFs, Word, Excel, etc.), and even perform web searches when needed.  

I'm here to make your life easier—so feel free to ask me anything! How can I assist you today? 😊


In [11]:
prompt_template = ChatPromptTemplate([
    ("system", "you are a helpful assistant, please answer question based on what user asked"),
    ("user", "This is user's question: {topic}, please answer yes or no")
])

# build a chain: prompt + model + parser
bool_qa_chain = prompt_template | model | StrOutputParser()
question = "1 + 1 > 2?"
result = bool_qa_chain.invoke({"topic": question})
print(result)

no


In [12]:
bool_qa_chain = prompt_template | model | BooleanOutputParser()
question = "1 + 1 > 2?"
result = bool_qa_chain.invoke({"topic": question})
print(result)

False


In [13]:
question = "1 + 1 > 1?"
result = bool_qa_chain.invoke({"topic": question})
print(result)

True


In [14]:
schemas = [
    ResponseSchema(name="name", description="the name of user"),
    ResponseSchema(name="age", description="the age of user")
]
parser = StructuredOutputParser.from_response_schemas(schemas)

prompt = PromptTemplate.from_template(
    "please based on the content provided to extract user's information, and return in json format:"
    + "\n{input}\n\n{format_instructions}"
)

chain = (
    prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

result = chain.invoke({"input": "My name is Bryan Ng, 24 years old. I am a AI engineer"})

In [15]:
print(result)

{'name': 'Bryan Ng', 'age': '24'}


In [44]:
news_gen_prompt = PromptTemplate.from_template(
    "please based on the newspaper title provided, please write a short paragraph"
    + "about the news content(about 100 words): \n\n title:{title}"
)

news_chain = news_gen_prompt | model | StrOutputParser()

schemas = [
    ResponseSchema(name="time", description="when the event happen"),
    ResponseSchema(name="location", description="where the event happen"),
    ResponseSchema(name="event", description="what is the event, please be concise")
]
parser = StructuredOutputParser.from_response_schemas(schemas)

summary_prompt = PromptTemplate.from_template(
    "please extract key information from the news content, and return it in json format"
    + "\n\n{news}\n\n{format_instructions}"
)

summary_chain = (
    summary_prompt.partial(format_instructions=parser.get_format_instructions())
    | model
    | parser
)

full_chain = news_chain | summary_chain

result = full_chain.invoke({"title": "Apple AirPods Pro 3 will be released on 19th, September in Seattle"})
print(result)

{'time': 'September 19th', 'location': 'Seattle', 'event': 'Apple AirPods Pro 3 launch event'}


In [46]:
def debug_print(x):
    print("result:", x)
    return x

debug_node = RunnableLambda(debug_print)

full_chain = news_chain | debug_node | summary_chain
result = full_chain.invoke({"title": "Apple AirPods Pro 3 will be released on 19th, September in Seattle"})
print(result)

result: Based on the announcement, Apple will unveil its highly anticipated AirPods Pro 3 on September 19th at a special event in Seattle. The next generation of the popular wireless earbuds is expected to feature significant upgrades, including enhanced active noise cancellation for an even more immersive listening experience and improved transparency mode. Industry analysts also predict the integration of new health monitoring sensors, potentially tracking wellness metrics like body temperature. The launch event in Seattle marks a notable departure from Apple's typical California-based announcements, generating additional excitement for the new product's debut.
{'time': 'September 19th', 'location': 'Seattle', 'event': 'Apple AirPods Pro 3 launch event'}


In [54]:
chatbot_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="You are hoshiro, is a helpful assistant."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [60]:
basic_qa_chain = chatbot_prompt | model | StrOutputParser()
messages_list = [
    HumanMessage(content="Hi there, I am Bryan, nice to meet you."),
    AIMessage(content="Hi, I am hoshiro, is a helpful assistant, nice to meet you too!"),
]
question = "Hello, what is my name?"
messages_list.append(HumanMessage(content=question))

In [64]:
result = basic_qa_chain.invoke({"messages": messages_list})
print(result)

Your name is Bryan! It's nice to meet you again. 😊
